# Tutorial  de Python

Link: https://www.youtube.com/watch?v=3dt4OGnU5sM

__[Código de Markdown para Jupyter](http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html)__

## Benchamark map vs comprehension lists
Fuente: https://stackoverflow.com/questions/1247486/list-comprehension-vs-map

### Map vs listas por comprension (python3)
Lo que estaba en stackoverflow esta mal comparado:
```python
$ python -mtimeit -s'xs=range(10)' 'map(hex, xs)'
1000000 loops, best of 3: 0.891 usec per loop
$ python -mtimeit -s'xs=range(10)' '[hex(x) for x in xs]'
1000000 loops, best of 3: 1.09 usec per loop
```
**Comparando lista (map) vs lista (comprension)**
```python
$ python -mtimeit -s'xs=range(10)' 'list(map(hex, xs))'
1000000 loops, best of 3: 1.19 usec per loop
$ python -mtimeit -s'xs=range(10)' '[hex(x) for x in xs]'
1000000 loops, best of 3: 1.04 usec per loop
```
**Comparando generador (map) vs generador(comprension)**
```python
$ python -mtimeit -s'xs=range(10)' 'map(hex, xs)'
1000000 loops, best of 3: 0.904 usec per loop
$ python -mtimeit -s'xs=range(10)' '(hex(x) for x in xs)'
1000000 loops, best of 3: 0.354 usec per loop
```

**Comparando lista (map/lambda) vs lista (comprension)**
```python
$ python -mtimeit -s'xs=range(10)' 'list(map(lambda x: x+2, xs))'
1000000 loops, best of 3: 1.47 usec per loop
$ python -mtimeit -s'xs=range(10)' '[x+2 for x in xs]'
1000000 loops, best of 3: 0.671 usec per loop
```

### Conclusion: Listas por comprension parecen ser más performantes que map o map/lambda
Pero cuidado: Hay que chequear en cada caso, ver los ejemplos de abajo.

In [114]:
!python -mtimeit -s'xs=range(3000)' 'list(map(hex, xs))'

1000 loops, best of 3: 235 usec per loop


In [115]:
!python -mtimeit -s'xs=range(3000)' '[hex(x) for x in xs]'

1000 loops, best of 3: 257 usec per loop


## Operador 'reduce' vs listas por comprension
(Benchmark)

El resultado es contundente !

En este caso claramente gana una **lista por comprension concatenada (usando generator)**.

Pero aún es mucho mejor usar <span style="color:blue">*itertools* junto con generators</span> !! (reduce los tiempos a la mitad con respecto a LC).

Lo más eficiente Cambiar *list_of_lists* por *gen_of_lists* 

Tambien puse un ejemplo de for tradicionales. Se ve que tarda un poco más que LC pero sigue siendo mucho menor que `reduce + add`

link: https://stackoverflow.com/questions/11264684/flatten-list-of-lists/11264799

In [84]:
# list_of_lists
iab_string = 'IAB=100001:1|IAB=200002:2|IAB=300003:3'
[[x[0]] * int(x[1]) for x in re.findall(r"IAB=(\d{6}):(\d+)", iab_string)]

[['100001'], ['200002', '200002'], ['300003', '300003', '300003']]

In [38]:
%%time
import re
from functools import reduce
from operator import add

iab_string = 'IAB=100001:1|IAB=200002:2|IAB=300003:3' * 30000

l = reduce(add, [[x[0]] * int(x[1]) for x in re.findall(r"IAB=(\d{6}):(\d+)", iab_string)])
                
print(l[0:7])
print(f"{len(l):,}")         

['100001', '200002', '200002', '300003', '300003', '300003', '100001']
180,000
CPU times: user 29.2 s, sys: 451 ms, total: 29.6 s
Wall time: 29.7 s


### Usando LC y ciclo for tradicional
Se observa que tarda **muchisimo menos que la opción anterior !!**

Usando un ciclo For para crear la lista:

In [5]:
%%time
import re

iab_string = 'IAB=100001:1|IAB=200002:2|IAB=300003:3' * 400000

list_of_lists = [[x[0]] * int(x[1]) for x in re.findall(r"IAB=(\d{6}):(\d+)", iab_string)]

l = []
for sublist in list_of_lists:
    for val in sublist:
        l.append(val)
        
print(l[0:7])
print(f"{len(l):,}")

['100001', '200002', '200002', '300003', '300003', '300003', '100001']
2,400,000
CPU times: user 2.12 s, sys: 124 ms, total: 2.24 s
Wall time: 2.25 s


**Usando lista por comprension mejora el tiempo**

In [6]:
%%time
import re

iab_string = 'IAB=100001:1|IAB=200002:2|IAB=300003:3' * 400000

list_of_lists = [[x[0]] * int(x[1]) for x in re.findall(r"IAB=(\d{6}):(\d+)", iab_string)]
#gen_of_lists = ([x[0]] * int(x[1]) for x in re.findall(r"IAB=(\d{6}):(\d+)", iab_string))
l = [val for sublist in list_of_lists for val in sublist]

print(l[0:7])
print(f"{len(l):,}")

['100001', '200002', '200002', '300003', '300003', '300003', '100001']
2,400,000
CPU times: user 1.76 s, sys: 123 ms, total: 1.88 s
Wall time: 1.88 s


**Usando lista por comprension + generator (en lugar de lista) mejora el tiempo aún más !!**

In [3]:
%%time
import re

iab_string = 'IAB=100001:1|IAB=200002:2|IAB=300003:3' * 400000

#list_of_lists = [[x[0]] * int(x[1]) for x in re.findall(r"IAB=(\d{6}):(\d+)", iab_string)]
gen_of_lists = ([x[0]] * int(x[1]) for x in re.findall(r"IAB=(\d{6}):(\d+)", iab_string))
l = [val for sublist in gen_of_lists for val in sublist]

print(l[0:7])
print(f"{len(l):,}")

['100001', '200002', '200002', '300003', '300003', '300003', '100001']
2,400,000
CPU times: user 1.04 s, sys: 59.7 ms, total: 1.1 s
Wall time: 1.11 s


### Usando itertools
Esta opción es **clara y eficiente !**

Si cuando creo 'list_of_list' lo hago un *generator* => el resultado es aún mejor !!!

In [2]:
%%time
import re
from itertools import chain

iab_string = 'IAB=100001:1|IAB=200002:2|IAB=300003:3' * 400000

#list_of_lists = [[x[0]] * int(x[1]) for x in re.findall(r"IAB=(\d{6}):(\d+)", iab_string)]
# Haciendo 'list_of_list' un generator funciona mucho más eficiente !!
gen_of_lists = ([x[0]] * int(x[1]) for x in re.findall(r"IAB=(\d{6}):(\d+)", iab_string))
l = list(chain.from_iterable(gen_of_lists))


print(l[0:7])
print(f"{len(l):,}")        

['100001', '200002', '200002', '300003', '300003', '300003', '100001']
2,400,000
CPU times: user 1.16 s, sys: 113 ms, total: 1.27 s
Wall time: 1.27 s


# Lists

In [2]:
#Lists
lista = [0,1,2,3,4,5]

lista

[0, 1, 2, 3, 4, 5]

In [9]:
print(lista[2:4])  #imprime posición 2 y 3, no incluye la pos 4.
print(lista[-2])

[2, 3]
4


## Listas por comprension
### Obtener cada elemento de la lista al cuadrado usando comprehension

In [10]:
squared_list = [n*n for n in lista]
squared_list

[0, 1, 4, 9, 16, 25]

### Obtener todos los numeros de la lista que sean pares

In [12]:
pares = [n for n in lista if n%2 == 0]
pares

[0, 2, 4]

### Crear pares (letra, numero) para 'abcd' y '0123' quedando: (a,0) (a,1) (a,2) ...(d,2)(d,3)(d,4)
Devolverá una tupla de letra y número

In [47]:
par_letra_numero = [(l,n) for l in 'abcd' for n in range(4)]
par_letra_numero

[('a', 0),
 ('a', 1),
 ('a', 2),
 ('a', 3),
 ('b', 0),
 ('b', 1),
 ('b', 2),
 ('b', 3),
 ('c', 0),
 ('c', 1),
 ('c', 2),
 ('c', 3),
 ('d', 0),
 ('d', 1),
 ('d', 2),
 ('d', 3)]

### Ejemplos con Map
Link: https://www.youtube.com/watch?v=hUes6y2b--0&list=WL&index=11&t=0s

#### Performnance
He comparado
* For tradicional
* Lista por comp
* Tupla por comp
* Map + funcion
* Map + lambda

El más rápido ha sido la *tupla por comp*, pero por muy poco margen. No hay un claro ganador.

In [56]:
n = 5000000

In [57]:
%%time
from math import pi

# Forma tradicional
radii = [1,2,3,4,5] * n
area = []
for r in radii:
    area.append(round(pi*r**2,2))
area

print(area[:4])
print(f"{len(area):,}")

[3.14, 12.57, 28.27, 50.27]
25,000,000
CPU times: user 21.9 s, sys: 335 ms, total: 22.2 s
Wall time: 22.3 s


In [58]:
%%time
from math import pi

radii = [1,2,3,4,5] * n
# Listas x comprension
area = [round(pi*r**2,2) for r in radii]

print(area[:4])
print(f"{len(area):,}")

[3.14, 12.57, 28.27, 50.27]
25,000,000
CPU times: user 19.8 s, sys: 492 ms, total: 20.3 s
Wall time: 20.4 s


In [61]:
%%time
from math import pi

radii = (1,2,3,4,5) * n
# tupla x comprension
area = [round(pi*r**2,2) for r in radii]

print(area[:4])
print(f"{len(area):,}")

[3.14, 12.57, 28.27, 50.27]
25,000,000
CPU times: user 19.5 s, sys: 458 ms, total: 20 s
Wall time: 20 s


In [59]:
%%time
from math import pi

radii = [1,2,3,4,5] * n
# Map + funcion
def calcular_area(r):
    return round(pi*r**2,2)

area = list(map(calcular_area,radii))

print(area[:4])
print(f"{len(area):,}")

[3.14, 12.57, 28.27, 50.27]
25,000,000
CPU times: user 20.5 s, sys: 450 ms, total: 20.9 s
Wall time: 21 s


In [ ]:
python -mtimeit -s'xs=range(10)' 'map(hex, xs)'

In [60]:
%%time
from math import pi

radii = [1,2,3,4,5] * n
# Map + lambda
area = list(map(lambda r:round(pi*r**2,2),radii))

print(area[:4])
print(f"{len(area):,}")

[3.14, 12.57, 28.27, 50.27]
25,000,000
CPU times: user 21 s, sys: 541 ms, total: 21.6 s
Wall time: 21.6 s


# Tuples

Una tupla está representada por valores separados por comas. **Las tuplas no se pueden cambiar** y la salida es entre paréntesis. Tambien pueden contener ciertos datos que SI pueden cambiar (??)

Debido a su inmutabilidad son más rápidas para el procesamiento comparadas con las listas. Por lo tanto, si la lista no cambia es mejor usar tuplas.

In [33]:
tuple_example = 0, 1, 4, 9, 16, 25
tuple_example

(0, 1, 4, 9, 16, 25)

In [34]:
tuple_example[2]

4

In [35]:
tuple_example[2] = 6      #Al ser ininmutables esto debería dar error

TypeError: 'tuple' object does not support item assignment

## Diccionarios

Diccionarios son datos de *key:value* desordenados con la condición de que los 'key' deben ser únicos dentro del mismo diccionario. Un par de llaves: {}

In [25]:
ext = {'geler':510, 'radriz':550,'nechi':543,'raul':500}
ext

{'geler': 510, 'radriz': 550, 'nechi': 543, 'raul': 500}

In [7]:
ext['nechi'] = 547     #Cambiar un dato
ext

{'geler': 510, 'nechi': 547, 'radriz': 550, 'raul': 500}

In [26]:
ext.keys()       # mostrar las keys

dict_keys(['geler', 'radriz', 'nechi', 'raul'])

In [27]:
ext.values()     # mostrar valores

dict_values([510, 550, 543, 500])

In [16]:
ext['geler']      #Acceder a un dato

510

### Ejemplos
Dados dos listas de string, las puedo unir 1 a 1 usando el comando *zip*

In [30]:
nombres = ['bruce', 'clark', 'peter']
heroes = ['batman', 'superman', 'hombre arania']

lista_de_tuplas = list(zip(nombres, heroes))
lista_de_tuplas

lista_de_tuplas[0][1]

'batman'

Ahora, quiero un diccionario de lo anterior donde 'key' sea el nombre y 'value' sea el heroe.

In [35]:
# Forma tradicional
my_dict = {}
for n in lista_de_tuplas:
    # my_dict[key] = value
    my_dict[n[0]] = n[1]
my_dict

{'bruce': 'batman', 'clark': 'superman', 'peter': 'hombre arania'}

In [36]:
# Forma mas compacta
my_dict = {}
for nombre,heroe in list(zip(nombres, heroes)):
    my_dict[nombre] = heroe
my_dict

{'bruce': 'batman', 'clark': 'superman', 'peter': 'hombre arania'}

In [40]:
# Usando diccionarios por comprension y excluyendo a peter
mi_dict = {nombre:heroe for nombre,heroe in list(zip(nombres, heroes)) if nombre != 'peter'}
mi_dict

{'bruce': 'batman', 'clark': 'superman'}

## SETs
Sets son como las listas pero tienen solamente **valores únicos y ordenados**

In [44]:
nums = [1,9,1,8,3,7,4,6,5,6,4,7,3,8,2,9,2,0,2,0,2,9,8,3,8,4,7]
my_set = set()
for n in nums:
    my_set.add(n)
my_set

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [46]:
# Estas dos formas funcionan -> se especifica que es un 'set' con '{}' sin ':' (como un dicts)
# mi_set = set(n for n in nums)
mi_set = {n for n in nums}
mi_set

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}